#### **Prepare Data**

##### **Install Packages and Library**

In [4]:
# Install Packages
%pip install selenium
%pip install webdriver-manager
%pip install nbformat>=4.2.0

# Manipulate
import datetime as dt
import pandas as pd
import numpy as np
import os
from selenium import webdriver

# Plot graph
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Finance
import yfinance as yf

# Install wget
!apt-get install wget

# Download the chromedriver.exe from GitHub
os.system('wget https://github.com/SamapanThongmee/Scrap_SET50_Index_Options_Data/blob/main/chromedriver.exe')

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


##### **SET50 Index Futures Data**

In [5]:
def getFutures(symbol='S50U23'): # H > M > U > Z
    futures = pd.DataFrame(columns=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI'])

    try:
        for page in range(15):
            url = f'https://classic.tfex.co.th/tfex/historicalTrading.html?symbol={symbol}&series=&page={page}&periodView=A&locale=en_US'
            driver.get(url)
            data = driver.page_source
            scrap = pd.read_html(data)[0]
            mask = scrap['Date'] == 'Grand Total'
            indices = int(scrap.index[mask][0])
            futures_data = scrap.iloc[:indices, :][['Date', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']].replace('-', 0)
            futures_data['Date'] = pd.to_datetime(futures_data['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
            futures_data['SP'] = pd.to_numeric(futures_data['SP'])
            futures_data['Vol'] = pd.to_numeric(futures_data['Vol'])
            futures_data['OI'] = pd.to_numeric(futures_data['OI'])
            futures_data['Symbol'] = symbol
            futures = pd.concat([futures, futures_data], axis=0)
            futures = futures.drop_duplicates(subset=['Date'], keep='last')

    except Exception as e:
        print(f"An error occurred: {e}")

    futures = futures[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']]
    futures = futures.sort_values(by='Date').reset_index(drop=True)

    return futures

S50H22 = getFutures(symbol='S50H22')
S50M22 = getFutures(symbol='S50M22')
S50U22 = getFutures(symbol='S50U22')
S50Z22 = getFutures(symbol='S50Z22')
S50H23 = getFutures(symbol='S50H23')
S50M23 = getFutures(symbol='S50M23')
S50U23 = getFutures(symbol='S50U23')
S50Z23 = getFutures(symbol='S50Z23')

Futures = pd.concat([S50H22, S50M22, S50U22, S50Z22,
                     S50H23, S50M23, S50U23, S50Z23],
                    axis=0).drop_duplicates(subset=['Date'], keep='last').reset_index(drop=True)
Futures['log_return'] = np.log(1 + Futures['Close'].pct_change())
Futures = Futures.dropna().reset_index(drop=True)
Futures

,Date,Symbol,Open,High,Low,Close,SP,Vol,OI,log_return
0,2021-03-31,S50H22,963.0,963.0,958.0,958.0,958.3,301,213.0,-0.004686
1,2021-04-01,S50H22,960.1,965.9,959.0,962.2,961.9,532,404.0,0.004375
2,2021-04-02,S50H22,965.0,967.4,962.0,964.8,964.7,479,608.0,0.002698
3,2021-04-05,S50H22,966.2,966.2,950.9,955.0,955.1,701,829.0,-0.010209
4,2021-04-07,S50H22,948.8,951.2,939.2,939.2,939.9,1098,1237.0,-0.016683
...,...,...,...,...,...,...,...,...,...,...
637,2023-11-17,S50Z23,877.5,884.3,872.7,877.7,877.8,239392,569366.0,-0.003299
638,2023-11-20,S50Z23,878.7,881.3,873.5,881.0,880.9,163295,541160.0,0.003753
639,2023-11-21,S50Z23,883.4,885.5,878.0,880.9,881.0,168464,536077.0,-0.000114
640,2023-11-22,S50Z23,880.1,881.8,870.3,874.5,875.0,222981,568693.0,-0.007292


##### **SET50 Market Breadth Data**

###### **Retrieve Data**

In [6]:
# Function to get date from Yahoo Finance
def get_data(ticker_list):
    start = dt.datetime.today() - dt.timedelta(days=(365 * 3))
    end = dt.datetime.today() + dt.timedelta(hours=7)
    high = pd.DataFrame()
    low = pd.DataFrame()
    adj_close = pd.DataFrame()

    try:
        for ticker in ticker_list:
            data = yf.download(ticker, start, end)
            high[ticker] = data['High']
            low[ticker] = data['Low']
            adj_close[ticker] = data['Adj Close']
    except Exception as e:
        print(f"An error occurred: {e}")

    return high, low, adj_close

# Ticker Lists
TICKER_2023H2 = ['ADVANC.BK', 'AOT.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BDMS.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'EA.BK', 'EGCO.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'HMPRO.BK', 'INTUCH.BK', 'IVL.BK', 'KBANK.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MINT.BK', 'MTC.BK', 'OR.BK', 'OSP.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'RATCH.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'TIDLOR.BK', 'TISCO.BK', 'TLI.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK', 'WHA.BK']
TICKER_2023H1 = ['ADVANC.BK', 'AOT.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BDMS.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'EA.BK', 'EGCO.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'HMPRO.BK', 'INTUCH.BK', 'IVL.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MINT.BK', 'MTC.BK', 'OR.BK', 'OSP.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'RATCH.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'TIDLOR.BK', 'TISCO.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK']
TICKER_2022H2 = ['ADVANC.BK', 'AOT.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BDMS.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BLA.BK', 'BTS.BK', 'CBG.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'EA.BK', 'EGCO.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MINT.BK', 'MTC.BK', 'OR.BK', 'OSP.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'TIDLOR.BK', 'TISCO.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK']
TICKER_2022H1 = ['ADVANC.BK', 'AOT.BK', 'BBL.BK', 'BDMS.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BJC.BK', 'BLA.BK', 'BTS.BK', 'CBG.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'DELTA.BK', 'EA.BK', 'EGCO.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MINT.BK', 'MTC.BK', 'OR.BK', 'OSP.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'STA.BK', 'TISCO.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK']

# Fetch data for each period
data_TICKER_2023H2 = get_data(TICKER_2023H2)
data_TICKER_2023H1 = get_data(TICKER_2023H1)
data_TICKER_2022H2 = get_data(TICKER_2022H2)
data_TICKER_2022H1 = get_data(TICKER_2022H1)

# Reset index for each DataFrame
HIGH_2023H2, LOW_2023H2, CLOSE_2023H2 = map(lambda x: x.reset_index(), data_TICKER_2023H2)
HIGH_2023H1, LOW_2023H1, CLOSE_2023H1 = map(lambda x: x.reset_index(), data_TICKER_2023H1)
HIGH_2022H2, LOW_2022H2, CLOSE_2022H2 = map(lambda x: x.reset_index(), data_TICKER_2022H2)
HIGH_2022H1, LOW_2022H1, CLOSE_2022H1 = map(lambda x: x.reset_index(), data_TICKER_2022H1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AOT.BK']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-11-24 22:04:05.573727 -> 2023-11-25 05:04:05.573727)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AWC.BK']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-11-24 22:04:05.573727 -> 2023-11-25 05:04:05.573727)')



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['CPALL.BK']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-11-24 22:04:05.573727 -> 2023-11-25 05:04:05.573727)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['EGCO.BK']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-11-24 22:04:05.573727 -> 2023-11-25 05:04:05.573727)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

###### **Create Function**

In [7]:
# New High - New Low
def getNewHighNewLow(High, Low):
    High = High.bfill().ffill()
    Low = Low.bfill().ffill()
    period_length = [20, 60, 200, 250]
    cut_rows = 250 #max(period_length)

    NewHigh = pd.DataFrame()
    NewHigh['Date'] = High['Date'].iloc[1:]
    High_without_date = High.drop(columns=['Date'])
    High_shifted = High_without_date.shift()

    number_of_stocks = 50

    for length in period_length:
        NewHigh['NH' + str(length)] = (High_without_date > High_shifted.rolling(length).max()).sum(axis=1).iloc[1:]

    NewLow = pd.DataFrame()
    NewLow['Date'] = Low['Date'].iloc[1:]
    Low_without_date = Low.drop(columns=['Date'])
    Low_shifted = Low_without_date.shift()

    for length in period_length:
        NewLow['NL' + str(length)] = (Low_without_date < Low_shifted.rolling(length).min()).sum(axis=1).iloc[1:]

    NHNL = NewHigh.merge(NewLow, on='Date', how='inner').tail(High.shape[0] - cut_rows).reset_index(drop=True)
    NHNL = NHNL.dropna().reset_index(drop=True)

    return NHNL

# Moving Average
def getMovingAvg(Close):
    Close = Close.bfill().ffill()
    ma_length = [20, 60, 200]
    cut_rows = 200 #max(ma_length)

    moving_avg_columns = ['MA' + str(length) for length in ma_length]
    moving_avg_values = pd.DataFrame()

    moving_avg_values['Date'] = Close['Date']

    for length in ma_length:
        moving_avg_values['MA' + str(length)] = (Close.drop(columns=['Date']) > Close.drop(columns=['Date']).rolling(length).mean()).sum(axis=1)

    MovingAvg = moving_avg_values[moving_avg_columns].iloc[cut_rows:,:]
    MovingAvg['Date'] = moving_avg_values['Date']
    MovingAvg = MovingAvg[['Date', 'MA'+str(ma_length[0]), 'MA'+str(ma_length[1]), 'MA'+str(ma_length[2])]].reset_index(drop=True)

    return MovingAvg

# Advance, Unchanged, and Declines
def getAdvUncDec(Close):
    data_close = Close.fillna(method='ffill').fillna(method='bfill')
    data_close['Date'] = pd.to_datetime(data_close['Date'])

    close_pct_change = data_close.drop('Date', axis=1).pct_change().dropna()

    Advances = (close_pct_change > 0).sum(axis=1)
    Unchanged = (close_pct_change == 0).sum(axis=1)
    Declines = (close_pct_change < 0).sum(axis=1)

    Adv_Unc_Dec = pd.DataFrame({
        'Date': data_close['Date'][1:],
        'Advance': Advances,
        'Unchanged': Unchanged,
        'Declines': Declines
    }).reset_index(drop=True)

    def getEMA(x, n):
        alpha = 2 / (1 + n)
        y = np.zeros_like(x)
        y[0] = x[0]
        for i in range(1, len(x)):
            y[i] = alpha * x[i] + (1 - alpha) * y[i - 1]
        return y

    Adv_Unc_Dec['AdvDev'] = Adv_Unc_Dec['Advance'] - Adv_Unc_Dec['Declines']

    def calculate_ema_columns(data, column_name, n, weight):
        ema = getEMA(data[column_name], n)
        data[column_name + str(n) + 'dEMA'] = ema
        ema_series = pd.Series(ema, index=data.index)
        data[column_name + str(n) + '-dEMA'] = data[column_name] * weight + ema_series.shift()

    calculate_ema_columns(Adv_Unc_Dec, 'AdvDev', 19, 0.10)
    calculate_ema_columns(Adv_Unc_Dec, 'AdvDev', 39, 0.05)

    Adv_Unc_Dec['McClellanOscillator'] = Adv_Unc_Dec['AdvDev19-dEMA'] - Adv_Unc_Dec['AdvDev39-dEMA']
    Adv_Unc_Dec = Adv_Unc_Dec[['Date', 'Advance', 'Unchanged', 'Declines', 'AdvDev', 'McClellanOscillator']].dropna().reset_index(drop=True)

    return Adv_Unc_Dec

###### **Market Breadth Data**

In [8]:
NHNL_2023H2 = getNewHighNewLow(HIGH_2023H2, LOW_2023H2)
NHNL_2023H1 = getNewHighNewLow(HIGH_2023H1, LOW_2023H1)
NHNL_2022H2 = getNewHighNewLow(HIGH_2022H2, LOW_2022H2)
NHNL_2022H1 = getNewHighNewLow(HIGH_2022H1, LOW_2022H1)

NHNL_2023H2 = NHNL_2023H2[(NHNL_2023H2['Date'] >= '2023-07-01') & (NHNL_2023H2['Date'] <= '2023-12-31')].reset_index(drop=True)
NHNL_2023H1 = NHNL_2023H1[(NHNL_2023H1['Date'] >= '2023-01-01') & (NHNL_2023H1['Date'] <= '2023-06-30')].reset_index(drop=True)
NHNL_2022H2 = NHNL_2022H2[(NHNL_2022H2['Date'] >= '2022-07-01') & (NHNL_2022H2['Date'] <= '2022-12-31')].reset_index(drop=True)
NHNL_2022H1 = NHNL_2022H1[(NHNL_2022H1['Date'] >= '2022-01-01') & (NHNL_2022H1['Date'] <= '2022-06-30')].reset_index(drop=True)

SET50_NHNL = pd.concat([NHNL_2022H1, NHNL_2022H2, NHNL_2023H1, NHNL_2023H2], axis=0).sort_values(by='Date', ascending=True).drop_duplicates(subset='Date', keep='last').reset_index(drop=True)

MA_2023H2 = getMovingAvg(CLOSE_2023H2)
MA_2023H1 = getMovingAvg(CLOSE_2023H1)
MA_2022H2 = getMovingAvg(CLOSE_2022H2)
MA_2022H1 = getMovingAvg(CLOSE_2022H1)

MA_2023H2 = MA_2023H2[(MA_2023H2['Date'] >= '2023-07-01') & (MA_2023H2['Date'] <= '2023-12-31')].reset_index(drop=True)
MA_2023H1 = MA_2023H1[(MA_2023H1['Date'] >= '2023-01-01') & (MA_2023H1['Date'] <= '2023-06-30')].reset_index(drop=True)
MA_2022H2 = MA_2022H2[(MA_2022H2['Date'] >= '2022-07-01') & (MA_2022H2['Date'] <= '2022-12-31')].reset_index(drop=True)
MA_2022H1 = MA_2022H1[(MA_2022H1['Date'] >= '2022-01-01') & (MA_2022H1['Date'] <= '2022-06-30')].reset_index(drop=True)

SET50_MA = pd.concat([MA_2022H1, MA_2022H2, MA_2023H1, MA_2023H2], axis=0).sort_values(by='Date', ascending=True).drop_duplicates(subset='Date', keep='last').reset_index(drop=True)

ADVUNCDEC_2023H2 = getAdvUncDec(CLOSE_2023H2)
ADVUNCDEC_2023H1 = getAdvUncDec(CLOSE_2023H1)
ADVUNCDEC_2022H2 = getAdvUncDec(CLOSE_2022H2)
ADVUNCDEC_2022H1 = getAdvUncDec(CLOSE_2022H1)

ADVUNCDEC_2023H2 = ADVUNCDEC_2023H2[(ADVUNCDEC_2023H2['Date'] >= '2023-07-01') & (ADVUNCDEC_2023H2['Date'] <= '2023-12-31')].reset_index(drop=True)
ADVUNCDEC_2023H1 = ADVUNCDEC_2023H1[(ADVUNCDEC_2023H1['Date'] >= '2023-01-01') & (ADVUNCDEC_2023H1['Date'] <= '2023-06-30')].reset_index(drop=True)
ADVUNCDEC_2022H2 = ADVUNCDEC_2022H2[(ADVUNCDEC_2022H2['Date'] >= '2022-07-01') & (ADVUNCDEC_2022H2['Date'] <= '2022-12-31')].reset_index(drop=True)
ADVUNCDEC_2022H1 = ADVUNCDEC_2022H1[(ADVUNCDEC_2022H1['Date'] >= '2022-01-01') & (ADVUNCDEC_2022H1['Date'] <= '2022-06-30')].reset_index(drop=True)

SET50_ADVUNCDEC = pd.concat([ADVUNCDEC_2022H1, ADVUNCDEC_2022H2, ADVUNCDEC_2023H1, ADVUNCDEC_2023H2], axis=0).sort_values(by='Date', ascending=True).drop_duplicates(subset='Date', keep='last').reset_index(drop=True)

###### **Manipulate Data**

In [9]:
Futures['Date'] = pd.to_datetime(Futures['Date'])

Breadth = SET50_ADVUNCDEC.merge(SET50_MA, on='Date', how='inner')
Breadth = Breadth.merge(SET50_NHNL, on='Date', how='inner')
Breadth['Date'] = pd.to_datetime(Breadth['Date'])

SET50_FUTURES_BREADTH = Futures.merge(Breadth, on='Date', how='inner').sort_values(by='Date', ascending=True).drop_duplicates(subset='Date', keep='last').reset_index(drop=True)
SET50_FUTURES_BREADTH

,Date,Symbol,Open,High,Low,Close,SP,Vol,OI,log_return,...,MA60,MA200,NH20,NH60,NH200,NH250,NL20,NL60,NL200,NL250
0,2022-01-04,S50Z22,984.9,992.3,981.8,988.0,987.8,3147,1441.0,0.006295,...,25,28,14,6,4,4,1,0,0,0
1,2022-01-05,S50Z22,988.0,991.5,986.8,990.7,990.4,1553,1853.0,0.002729,...,25,31,18,6,4,4,1,1,0,0
2,2022-01-06,S50Z22,983.9,985.7,972.5,974.6,974.2,3514,3844.0,-0.016385,...,22,26,2,1,1,1,2,1,0,0
3,2022-01-07,S50Z22,975.4,976.5,970.5,972.2,972.6,1191,4072.0,-0.002466,...,24,25,3,2,1,1,1,0,0,0
4,2022-01-10,S50Z22,973.9,978.3,970.6,973.2,973.0,1281,4341.0,0.001028,...,24,25,5,2,1,1,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2023-06-26,S50Z23,913.0,916.9,906.0,906.6,906.7,17271,60540.0,-0.009223,...,11,12,1,0,0,0,14,10,8,6
358,2023-06-27,S50Z23,907.0,910.6,900.0,900.5,900.4,22825,66052.0,-0.006751,...,9,12,0,0,0,0,11,9,8,6
359,2023-06-28,S50Z23,902.3,906.5,892.0,892.7,892.6,20671,69804.0,-0.008700,...,6,12,0,0,0,0,13,9,6,5
360,2023-06-29,S50Z23,892.6,907.4,891.0,903.0,902.9,27681,72010.0,0.011472,...,8,13,1,1,0,0,11,6,6,5


##### **SET Market Breadth Data**

###### **Retrieve Data**

In [10]:
# Tickers
SET_TICKERS = ['2S.BK', '3K-BAT.BK', '7UP.BK', 'A.BK', 'AAI.BK', 'AAV.BK', 'ACC.BK', 'ACE.BK', 'ACG.BK', 'ADVANC.BK', 'AEONTS.BK', 'AFC.BK', 'AGE.BK', 'AH.BK', 'AHC.BK', 'AI.BK', 'AIE.BK', 'AIMCG.BK', 'AIMIRT.BK', 'AIT.BK', 'AJ.BK', 'AJA.BK', 'AKR.BK', 'AKS.BK', 'ALLA.BK', 'ALLY.BK', 'ALT.BK', 'ALUCON.BK', 'AMANAH.BK', 'AMARIN.BK', 'AMATA.BK', 'AMATAR.BK', 'AMATAV.BK', 'AMC.BK', 'AMR.BK', 'ANAN.BK', 'AOT.BK', 'AP.BK', 'APCO.BK', 'APCS.BK', 'APEX.BK', 'APURE.BK', 'AQUA.BK', 'AS.BK', 'ASAP.BK', 'ASEFA.BK', 'ASIA.BK', 'ASIAN.BK', 'ASIMAR.BK', 'ASK.BK', 'ASP.BK', 'ASW.BK', 'AURA.BK', 'AWC.BK', 'AYUD.BK', 'B.BK', 'B-WORK.BK', 'B52.BK', 'BA.BK', 'BAFS.BK', 'BAM.BK', 'BANPU.BK', 'BAREIT.BK', 'BAY.BK', 'BBGI.BK', 'BBL.BK', 'BCH.BK', 'BCP.BK', 'BCPG.BK', 'BCT.BK', 'BDMS.BK', 'BEAUTY.BK', 'BEC.BK', 'BEM.BK', 'BEYOND.BK', 'BGC.BK', 'BGRIM.BK', 'BH.BK', 'BIG.BK', 'BIOTEC.BK', 'BIZ.BK', 'BJC.BK', 'BJCHI.BK', 'BKD.BK', 'BKI.BK', 'BKKCP.BK', 'BLA.BK', 'BLAND.BK', 'BLC.BK', 'BLISS.BK', 'BOFFICE.BK', 'BPP.BK', 'BR.BK', 'BRI.BK', 'BROCK.BK', 'BRR.BK', 'BRRGIF.BK', 'BSBM.BK', 'BTG.BK', 'BTNC.BK', 'BTS.BK', 'BTSGIF.BK', 'BUI.BK', 'BWG.BK', 'BYD.BK', 'CBG.BK', 'CCET.BK', 'CCP.BK', 'CEN.BK', 'CENTEL.BK', 'CFRESH.BK', 'CGD.BK', 'CGH.BK', 'CH.BK', 'CHARAN.BK', 'CHASE.BK', 'CHAYO.BK', 'CHG.BK', 'CHOTI.BK', 'CI.BK', 'CIMBT.BK', 'CITY.BK', 'CIVIL.BK', 'CK.BK', 'CKP.BK', 'CM.BK', 'CMAN.BK', 'CMC.BK', 'CMR.BK', 'CNT.BK', 'COCOCO.BK', 'COM7.BK', 'COTTO.BK', 'CPALL.BK', 'CPAXT.BK', 'CPF.BK', 'CPH.BK', 'CPI.BK', 'CPL.BK', 'CPN.BK', 'CPNCG.BK', 'CPNREIT.BK', 'CPT.BK', 'CPTGF.BK', 'CPW.BK', 'CRANE.BK', 'CRC.BK', 'CSC.BK', 'CSP.BK', 'CSR.BK', 'CSS.BK', 'CTARAF.BK', 'CTW.BK', 'CV.BK', 'CWT.BK', 'DCC.BK', 'DCON.BK', 'DDD.BK', 'DELTA.BK', 'DEMCO.BK', 'DIF.BK', 'DMT.BK', 'DOHOME.BK', 'DREIT.BK', 'DRT.BK', 'DTCENT.BK', 'DTCI.BK', 'DUSIT.BK', 'EA.BK', 'EASON.BK', 'EASTW.BK', 'ECL.BK', 'EE.BK', 'EGATIF.BK', 'EGCO.BK', 'EKH.BK', 'EMC.BK', 'EP.BK', 'EPG.BK', 'ERW.BK', 'ERWPF.BK', 'ESSO.BK', 'ESTAR.BK', 'ETC.BK', 'EVER.BK', 'F&D.BK', 'FANCY.BK', 'FE.BK', 'FMT.BK', 'FN.BK', 'FNS.BK', 'FORTH.BK', 'FPT.BK', 'FSX.BK', 'FTE.BK', 'FTI.BK', 'FTREIT.BK', 'FUTUREPF.BK', 'GABLE.BK', 'GAHREIT.BK', 'GBX.BK', 'GC.BK', 'GEL.BK', 'GENCO.BK', 'GFPT.BK', 'GGC.BK', 'GIFT.BK', 'GJS.BK', 'GL.BK', 'GLAND.BK', 'GLOBAL.BK', 'GLOCON.BK', 'GPI.BK', 'GPSC.BK', 'GRAMMY.BK', 'GRAND.BK', 'GREEN.BK', 'GROREIT.BK', 'GSTEEL.BK', 'GULF.BK', 'GUNKUL.BK', 'GVREIT.BK', 'GYT.BK', 'HANA.BK', 'HENG.BK', 'HFT.BK', 'HMPRO.BK', 'HPF.BK', 'HTC.BK', 'HTECH.BK', 'HUMAN.BK', 'HYDROGEN.BK', 'ICC.BK', 'ICHI.BK', 'ICN.BK', 'IFEC.BK', 'IFS.BK', 'IHL.BK', 'III.BK', 'ILINK.BK', 'ILM.BK', 'IMPACT.BK', 'INET.BK', 'INETREIT.BK', 'INGRS.BK', 'INOX.BK', 'INSET.BK', 'INSURE.BK', 'INTUCH.BK', 'IRC.BK', 'IRPC.BK', 'IT.BK', 'ITC.BK', 'ITD.BK', 'ITEL.BK', 'IVL.BK', 'J.BK', 'JAS.BK', 'JASIF.BK', 'JCK.BK', 'JCT.BK', 'JDF.BK', 'JKN.BK', 'JMART.BK', 'JMT.BK', 'JR.BK', 'JTS.BK', 'KAMART.BK', 'KBANK.BK', 'KBS.BK', 'KBSPIF.BK', 'KC.BK', 'KCAR.BK', 'KCE.BK', 'KCG.BK', 'KDH.BK', 'KEX.BK', 'KGI.BK', 'KIAT.BK', 'KISS.BK', 'KKC.BK', 'KKP.BK', 'KPNPF.BK', 'KSL.BK', 'KTB.BK', 'KTBSTMR.BK', 'KTC.BK', 'KTIS.BK', 'KWC.BK', 'KWI.BK', 'KYE.BK', 'L&E.BK', 'LALIN.BK', 'LANNA.BK', 'LEE.BK', 'LH.BK', 'LHFG.BK', 'LHHOTEL.BK', 'LHK.BK', 'LHPF.BK', 'LHSC.BK', 'LOXLEY.BK', 'LPF.BK', 'LPH.BK', 'LPN.BK', 'LRH.BK', 'LST.BK', 'LUXF.BK', 'M.BK', 'M-CHAI.BK', 'M-II.BK', 'M-PAT.BK', 'M-STOR.BK', 'MACO.BK', 'MAJOR.BK', 'MALEE.BK', 'MANRIN.BK', 'MATCH.BK', 'MATI.BK', 'MAX.BK', 'MBK.BK', 'MC.BK', 'MCOT.BK', 'MCS.BK', 'MDX.BK', 'MEGA.BK', 'MENA.BK', 'METCO.BK', 'MFC.BK', 'MFEC.BK', 'MGC.BK', 'MICRO.BK', 'MIDA.BK', 'MILL.BK', 'MINT.BK', 'MIPF.BK', 'MIT.BK', 'MJD.BK', 'MJLF.BK', 'MK.BK', 'ML.BK', 'MNIT.BK', 'MNIT2.BK', 'MNRF.BK', 'MODERN.BK', 'MONO.BK', 'MOSHI.BK', 'MSC.BK', 'MST.BK', 'MTC.BK', 'MTI.BK', 'NATION.BK', 'NC.BK', 'NCAP.BK', 'NCH.BK', 'NEP.BK', 'NER.BK', 'NEW.BK', 'NEX.BK', 'NFC.BK', 'NKI.BK', 'NNCL.BK', 'NOBLE.BK', 'NOK.BK', 'NOVA.BK', 'NRF.BK', 'NSL.BK', 'NTV.BK', 'NUSA.BK', 'NV.BK', 'NVD.BK', 'NWR.BK', 'NYT.BK', 'OCC.BK', 'OGC.BK', 'OHTL.BK', 'ONEE.BK', 'OR.BK', 'ORI.BK', 'OSP.BK', 'PACE.BK', 'PAF.BK', 'PAP.BK', 'PATO.BK', 'PB.BK', 'PCC.BK', 'PCSGH.BK', 'PDJ.BK', 'PEACE.BK', 'PERM.BK', 'PF.BK', 'PG.BK', 'PHG.BK', 'PIN.BK', 'PJW.BK', 'PK.BK', 'PL.BK', 'PLANB.BK', 'PLAT.BK', 'PLE.BK', 'PLUS.BK', 'PM.BK', 'PMTA.BK', 'POLAR.BK', 'POLY.BK', 'POPF.BK', 'PORT.BK', 'POST.BK', 'PPF.BK', 'PPP.BK', 'PPPM.BK', 'PQS.BK', 'PR9.BK', 'PRAKIT.BK', 'PREB.BK', 'PRECHA.BK', 'PRG.BK', 'PRIME.BK', 'PRIN.BK', 'PRINC.BK', 'PRM.BK', 'PRO.BK', 'PROSPECT.BK', 'PRTR.BK', 'PSH.BK', 'PSL.BK', 'PSP.BK', 'PT.BK', 'PTECH.BK', 'PTG.BK', 'PTL.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'PYLON.BK', 'Q-CON.BK', 'QH.BK', 'QHHR.BK', 'QHOP.BK', 'QHPF.BK', 'QTC.BK', 'RABBIT.BK', 'RAM.BK', 'RATCH.BK', 'RBF.BK', 'RCL.BK', 'RICHY.BK', 'RJH.BK', 'RML.BK', 'ROCK.BK', 'ROH.BK', 'ROJNA.BK', 'RPC.BK', 'RPH.BK', 'RS.BK', 'RSP.BK', 'RT.BK', 'S.BK', 'S&J.BK', 'S11.BK', 'SA.BK', 'SABINA.BK', 'SABUY.BK', 'SAK.BK', 'SAM.BK', 'SAMART.BK', 'SAMCO.BK', 'SAMTEL.BK', 'SAPPE.BK', 'SAT.BK', 'SAUCE.BK', 'SAV.BK', 'SAWAD.BK', 'SAWANG.BK', 'SBNEXT.BK', 'SC.BK', 'SCAP.BK', 'SCB.BK', 'SCC.BK', 'SCCC.BK', 'SCG.BK', 'SCGP.BK', 'SCI.BK', 'SCM.BK', 'SCN.BK', 'SCP.BK', 'SDC.BK', 'SE-ED.BK', 'SEAFCO.BK', 'SEAOIL.BK', 'SENA.BK', 'SFLEX.BK', 'SGC.BK', 'SGP.BK', 'SHANG.BK', 'SHR.BK', 'SIAM.BK', 'SINGER.BK', 'SINO.BK', 'SIRI.BK', 'SIRIP.BK', 'SIS.BK', 'SISB.BK', 'SITHAI.BK', 'SJWD.BK', 'SKE.BK', 'SKN.BK', 'SKR.BK', 'SKY.BK', 'SLP.BK', 'SM.BK', 'SMIT.BK', 'SMK.BK', 'SMPC.BK', 'SMT.BK', 'SNC.BK', 'SNNP.BK', 'SNP.BK', 'SO.BK', 'SOLAR.BK', 'SORKON.BK', 'SPACK.BK', 'SPALI.BK', 'SPC.BK', 'SPCG.BK', 'SPG.BK', 'SPI.BK', 'SPRC.BK', 'SPRIME.BK', 'SQ.BK', 'SRICHA.BK', 'SRIPANWA.BK', 'SSC.BK', 'SSF.BK', 'SSP.BK', 'SSPF.BK', 'SSSC.BK', 'SST.BK', 'SSTRT.BK', 'STA.BK', 'STANLY.BK', 'STARK.BK', 'STEC.BK', 'STECH.BK', 'STGT.BK', 'STHAI.BK', 'STI.BK', 'STPI.BK', 'SUC.BK', 'SUN.BK', 'SUPER.BK', 'SUPEREIF.BK', 'SUSCO.BK', 'SUTHA.BK', 'SVI.BK', 'SVOA.BK', 'SVT.BK', 'SYMC.BK', 'SYNEX.BK', 'SYNTEC.BK', 'TAE.BK', 'TAN.BK', 'TASCO.BK', 'TC.BK', 'TCAP.BK', 'TCC.BK', 'TCJ.BK', 'TCMC.BK', 'TCOAT.BK', 'TEAM.BK', 'TEAMG.BK', 'TEGH.BK', 'TEKA.BK', 'TFFIF.BK', 'TFG.BK', 'TFI.BK', 'TFM.BK', 'TFMAMA.BK', 'TGE.BK', 'TGH.BK', 'TGPRO.BK', 'TH.BK', 'THAI.BK', 'THANI.BK', 'THCOM.BK', 'THE.BK', 'THG.BK', 'THIP.BK', 'THRE.BK', 'THREL.BK', 'TIDLOR.BK', 'TIF1.BK', 'TIPCO.BK', 'TIPH.BK', 'TISCO.BK', 'TK.BK', 'TKC.BK', 'TKN.BK', 'TKS.BK', 'TKT.BK', 'TLHPF.BK', 'TLI.BK', 'TMD.BK', 'TMT.BK', 'TNITY.BK', 'TNL.BK', 'TNPC.BK', 'TNPF.BK', 'TNR.BK', 'TOA.BK', 'TOG.BK', 'TOP.BK', 'TOPP.BK', 'TPA.BK', 'TPAC.BK', 'TPBI.BK', 'TPCS.BK', 'TPIPL.BK', 'TPIPP.BK', 'TPOLY.BK', 'TPP.BK', 'TPRIME.BK', 'TQM.BK', 'TR.BK', 'TRC.BK', 'TRITN.BK', 'TRU.BK', 'TRUBB.BK', 'TRUE.BK', 'TSC.BK', 'TSE.BK', 'TSI.BK', 'TSTE.BK', 'TSTH.BK', 'TTA.BK', 'TTB.BK', 'TTCL.BK', 'TTI.BK', 'TTLPF.BK', 'TTT.BK', 'TTW.BK', 'TU.BK', 'TU-PF.BK', 'TVH.BK', 'TVO.BK', 'TWP.BK', 'TWPC.BK', 'TWZ.BK', 'TYCN.BK', 'UAC.BK', 'UBE.BK', 'UMI.BK', 'UNIQ.BK', 'UOBKH.BK', 'UP.BK', 'UPF.BK', 'UPOIC.BK', 'URBNPF.BK', 'UTP.BK', 'UV.BK', 'UVAN.BK', 'VARO.BK', 'VGI.BK', 'VIBHA.BK', 'VIH.BK', 'VNG.BK', 'VPO.BK', 'VRANDA.BK', 'W.BK', 'WACOAL.BK', 'WAVE.BK', 'WFX.BK', 'WGE.BK', 'WHA.BK', 'WHABT.BK', 'WHAIR.BK', 'WHART.BK', 'WHAUP.BK', 'WICE.BK', 'WIIK.BK', 'WIN.BK', 'WINDOW.BK', 'WORK.BK', 'WP.BK', 'WPH.BK', 'XPG.BK', 'ZAA.BK', 'ZEN.BK']
batches = [SET_TICKERS[i:i+50] for i in range(0, len(SET_TICKERS), 50)]

# Stock data
def getData(ticker_list):
    start = dt.datetime.today() - dt.timedelta(days=(365*2))
    end = dt.datetime.today() + dt.timedelta(hours=7)
    high = pd.DataFrame()
    low = pd.DataFrame()
    adj_close = pd.DataFrame()

    for ticker in ticker_list:
        data = yf.download(ticker, start, end)
        high[ticker] = data['High']
        low[ticker] = data['Low']
        adj_close[ticker] = data['Adj Close']

    return high, low, adj_close

# Create function
def getHLC():
    high_data = []
    low_data = []
    adj_close_data = []

    for batch in batches:
        data = getData(batch)
        high_data.append(data[0])
        low_data.append(data[1])
        adj_close_data.append(data[2])

    # Concatenate data
    concatenated_High = pd.concat(high_data, axis=1)
    concatenated_Low = pd.concat(low_data, axis=1)
    concatenated_Close = pd.concat(adj_close_data, axis=1)

    # Process data
    for df in [concatenated_High, concatenated_Low, concatenated_Close]:
        df = df[~df.index.duplicated(keep='first')]
        df = df.bfill(axis='rows')
        df = df.ffill(axis='rows')
        df = df.reset_index()

    return concatenated_High, concatenated_Low, concatenated_Close

# Pull data
High, Low, Close = getHLC()
column_rename_dict = {col: col.replace('.BK', '') for col in High.columns}

High.columns = High.columns.str.replace('.BK', '')
Low.columns = Low.columns.str.replace('.BK', '')
Close.columns = Close.columns.str.replace('.BK', '')

# Clean data
High = High.reset_index()
Low = Low.reset_index()
Close = Close.reset_index()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

C:\Users\samap\AppData\Local\Temp\ipykernel_27844\3794867829.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  High.columns = High.columns.str.replace('.BK', '')
C:\Users\samap\AppData\Local\Temp\ipykernel_27844\3794867829.py:52: FutureWarning: The default value of regex will change from True to False in a future version.
  Low.columns = Low.columns.str.replace('.BK', '')
C:\Users\samap\AppData\Local\Temp\ipykernel_27844\3794867829.py:53: FutureWarning: The default value of regex will change from True to False in a future version.
  Close.columns = Close.columns.str.replace('.BK', '')


###### **Create Function**

In [11]:
# Exponential Moving Average
def getEMA(x, n):
    alpha = 2/(1+n)
    y = np.zeros_like(x)
    y[0] = x[0]
    
    for i in range(1, len(x)):
        y[i] = alpha * x[i] + (1-alpha) * y[i-1]
    return y

# New High - New Low
def getNewHighNewLow(High, Low):
    High = High.bfill().ffill()
    Low = Low.bfill().ffill()
    period_length = [20, 60, 200, 250]
    cut_rows = 250 #max(period_length)

    NewHigh = pd.DataFrame()
    NewHigh['Date'] = High['Date'].iloc[1:]
    High_without_date = High.drop(columns=['Date'])
    High_shifted = High_without_date.shift()

    number_of_stocks = High.shape[1] - 1

    for length in period_length:
        NewHigh['NH' + str(length)] = round(100 * (High_without_date > High_shifted.rolling(length).max()).sum(axis=1).iloc[1:] / number_of_stocks, 2)
    
    NewLow = pd.DataFrame()
    NewLow['Date'] = Low['Date'].iloc[1:]
    Low_without_date = Low.drop(columns=['Date'])
    Low_shifted = Low_without_date.shift()

    for length in period_length:
        NewLow['NL' + str(length)] = round(100 * (Low_without_date < Low_shifted.rolling(length).min()).sum(axis=1).iloc[1:] / number_of_stocks, 2)

    NHNL = NewHigh.merge(NewLow, on='Date', how='inner').tail(High.shape[0] - cut_rows).reset_index(drop=True)
    
    NHNL['DiffNHNL20d'] = getEMA((NHNL['NH20'] - NHNL['NL20']), 5)
    NHNL['DiffNHNL60d'] = getEMA((NHNL['NH60'] - NHNL['NL60']), 5)
    NHNL['DiffNHNL250d'] = getEMA((NHNL['NH250'] - NHNL['NL250']), 5)
    NHNL['DiffNHNL20d'] = round(NHNL['DiffNHNL20d'], 2)
    NHNL['DiffNHNL60d'] = round(NHNL['DiffNHNL60d'], 2)
    NHNL['DiffNHNL250d'] = round(NHNL['DiffNHNL250d'], 2)

    NHNL = NHNL.dropna().reset_index(drop=True)
    NHNL = NHNL[['Date',
                 'NH20', 'NH60', 'NH250', 'NL20', 'NL60', 'NL250',
                 'DiffNHNL20d', 'DiffNHNL60d', 'DiffNHNL250d']]
    return NHNL

# Moving Average
def getMovingAvg(Close):
    Close = Close.bfill().ffill()
    ma_length = [20, 60, 200]
    cut_rows = 200 #max(ma_length)
    
    number_of_stocks = Close.shape[1] - 1

    moving_avg_columns = ['MA' + str(length) for length in ma_length]
    moving_avg_values = pd.DataFrame()

    moving_avg_values['Date'] = Close['Date']

    for length in ma_length:
        moving_avg_values['MA' + str(length)] = round(100 * (Close.drop(columns=['Date']) > Close.drop(columns=['Date']).rolling(length).mean()).sum(axis=1) / number_of_stocks, 2)

    MovingAvg = moving_avg_values[moving_avg_columns].iloc[cut_rows:,:]
    MovingAvg['Date'] = moving_avg_values['Date']
    MovingAvg = MovingAvg[['Date', 'MA'+str(ma_length[0]), 'MA'+str(ma_length[1]), 'MA'+str(ma_length[2])]].reset_index(drop=True)
    
    return MovingAvg

# Advance, Unchanged, and Declines
def getAdvUncDec(Close):
    data_close = Close.fillna(method='ffill').fillna(method='bfill')
    data_close['Date'] = pd.to_datetime(data_close['Date'])

    close_pct_change = data_close.drop('Date', axis=1).pct_change().dropna()
    
    Advances = (close_pct_change > 0).sum(axis=1)
    Unchanged = (close_pct_change == 0).sum(axis=1)
    Declines = (close_pct_change < 0).sum(axis=1)

    Adv_Unc_Dec = pd.DataFrame({
        'Date': data_close['Date'][1:],
        'Advance': Advances,
        'Unchanged': Unchanged,
        'Declines': Declines
    }).reset_index(drop=True)

    def getEMA(x, n):
        alpha = 2 / (1 + n)
        y = np.zeros_like(x)
        y[0] = x[0]
        for i in range(1, len(x)):
            y[i] = alpha * x[i] + (1 - alpha) * y[i - 1]
        return y

    Adv_Unc_Dec['AdvDev'] = Adv_Unc_Dec['Advance'] - Adv_Unc_Dec['Declines']

    def calculate_ema_columns(data, column_name, n, weight):
        ema = getEMA(data[column_name], n)
        data[column_name + str(n) + 'dEMA'] = ema
        ema_series = pd.Series(ema, index=data.index)
        data[column_name + str(n) + '-dEMA'] = data[column_name] * weight + ema_series.shift()
        
    calculate_ema_columns(Adv_Unc_Dec, 'AdvDev', 19, 0.10)
    calculate_ema_columns(Adv_Unc_Dec, 'AdvDev', 39, 0.05)
    
    Adv_Unc_Dec['McClellanOscillator'] = Adv_Unc_Dec['AdvDev19-dEMA'] - Adv_Unc_Dec['AdvDev39-dEMA']
    Adv_Unc_Dec = Adv_Unc_Dec[['Date', 'Advance', 'Unchanged', 'Declines', 'AdvDev', 'McClellanOscillator']].dropna().reset_index(drop=True)

    return Adv_Unc_Dec

# SET Index
def getSET():
    start = dt.datetime.today() - dt.timedelta((365*2))
    end = dt.datetime.today() + dt.timedelta(hours=7)
    SET = yf.download("^SET.BK",start,end).reset_index()[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    return SET

###### **Market Breadth Data**

In [12]:
# Manipulate data
NHNL = getNewHighNewLow(High, Low)
MA = getMovingAvg(Close)
AdvDec = getAdvUncDec(Close)

###### **Manipulate Data**

In [13]:
SET_Index = getSET()

# Combine data
SET_Index = SET_Index.merge(NHNL, on='Date', how='inner')
SET_Index = SET_Index.merge(MA, on='Date', how='inner')
SET_Index = SET_Index.merge(AdvDec, on='Date', how='inner')
SET_INDEX_BREADTH = SET_Index.dropna().reset_index(drop=True)

SET_INDEX_BREADTH

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume,NH20,NH60,NH250,NL20,...,DiffNHNL60d,DiffNHNL250d,MA20,MA60,MA200,Advance,Unchanged,Declines,AdvDev,McClellanOscillator
0,2022-12-08,1624.290039,1625.900024,1615.060059,1620.489990,3849800,4.07,1.60,0.87,6.69,...,-2.62,-1.46,41.42,37.94,35.47,215,295,178,37,-9.15
1,2022-12-09,1621.969971,1629.930054,1618.130005,1623.130005,3281100,3.92,1.74,1.31,2.91,...,-1.80,-0.97,42.01,37.79,35.90,206,277,205,1,-6.95
2,2022-12-13,1629.829956,1633.069946,1621.410034,1625.910034,4968900,4.51,2.47,1.16,5.52,...,-1.68,-1.28,43.75,38.23,34.59,192,229,267,-75,-8.75
3,2022-12-14,1630.489990,1638.880005,1628.170044,1633.359985,4761200,6.83,4.07,2.33,5.52,...,-1.12,-1.14,45.78,39.53,36.05,245,310,133,112,-2.40
4,2022-12-15,1632.640015,1634.020020,1616.989990,1620.280029,5614400,3.20,1.74,0.73,8.72,...,-2.15,-1.59,38.95,37.06,34.74,94,228,366,-272,-14.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2023-11-20,1415.680054,1419.859985,1409.550049,1419.439941,2779600,6.83,1.45,0.15,2.91,...,-2.73,-2.61,52.18,21.08,17.88,225,296,167,58,19.90
233,2023-11-21,1427.040039,1427.130005,1418.420044,1423.609985,2916400,7.56,1.16,0.00,1.45,...,-1.87,-2.03,56.83,22.53,18.17,234,307,147,87,23.35
234,2023-11-22,1422.020020,1423.069946,1407.369995,1414.150024,3381200,5.96,1.31,0.29,2.62,...,-1.59,-1.79,60.32,23.98,18.60,245,203,240,5,21.25
235,2023-11-23,1414.930054,1415.020020,1400.479980,1406.609985,3392000,5.67,1.31,0.44,2.18,...,-1.15,-1.48,58.43,23.55,19.04,186,262,240,-54,16.30


#### **Visualization**

##### **SET50 Visualization**

###### **SET50 Index Futures, Advances-Declines, McClellan Oscillator**

In [15]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET50_FUTURES_BREADTH['Date'].iloc[0], end=SET50_FUTURES_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET50_FUTURES_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET50_FUTURES_BREADTH['Date'],
                            open=SET50_FUTURES_BREADTH['Open'],
                            high=SET50_FUTURES_BREADTH['High'],
                            low=SET50_FUTURES_BREADTH['Low'],
                            close=SET50_FUTURES_BREADTH['Close'],
                            name='SET50 Index Futures'))

# Plot Advances, Unchanged, and Declines on the second row
fig1.add_trace(go.Bar(name="Advances", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['Advance'], 
                      marker_color='rgb(128,186,90)'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="Unchanged", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['Unchanged'], 
                      marker_color='rgb(204,204,204)'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="Declines", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['Declines'], 
                      marker_color='rgb(249,123,114)'),
              row=2, col=1)
fig1.update_layout(barmode='relative')

# Plot McClellan Oscillator on the third row
fig1.add_trace(go.Scatter(name='McClellan Oscillator',
                          x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['McClellanOscillator'],
                          mode='lines', line=dict(color='#496CFD', width=2)),
               row=3, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET50 Index Futures",
                  range=[800, 1000], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="Advances/Unchanged/Declines", 
                  range=[0, 50], tickmode='array', tickvals=(0, 50, 25),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="McClellan Oscillator", 
                  range=[-8, 8], tickmode='array', tickvals=(-8, 8, 0.0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=10, row=3, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET50 Index Futures and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([988. , 990.7, 974.6, ..., 892.7, 903. , 917. ]),
              'high': array([992.3, 991.5, 985.7, ..., 906.5, 907.4, 917.9]),
              'low': array([981.8, 986.8, 972.5, ..., 892. , 891. , 905.3]),
              'name': 'SET50 Index Futures',
              'open': array([984.9, 988. , 983.9, ..., 902.3, 892.6, 906.5]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0),
                          datetime.datetime(2022, 1, 6, 0, 0), ...,
                          datetime.datetime(2023, 6, 28, 0, 0),
                          datetime.datetime(2023, 6, 29, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object)},
             {'marker': {'color': 'rgb(128,186,90)'},
              'name': 'Advances',
              'type': 'bar',
              'x': array([datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0),
                          datetime.datetime(2022, 1, 6, 0, 0), ...,
                          datetime.datetime(2023, 6, 28, 0, 0),
                          datetime.datetime(2023, 6, 29, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object),
              'xaxis': 'x2',
              'y': array([34., 23.,  3., ..., 13., 28., 44.]),
              'yaxis': 'y2'},
             {'marker': {'color': 'rgb(204,204,204)'},
              'name': 'Unchanged',
              'type': 'bar',
              'x': array([datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0),
                          datetime.datetime(2022, 1, 6, 0, 0), ...,
                          datetime.datetime(2023, 6, 28, 0, 0),
                          datetime.datetime(2023, 6, 29, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object),
              'xaxis': 'x2',
              'y': array([ 5., 10.,  2., ...,  5.,  6.,  5.]),
              'yaxis': 'y2'},
             {'marker': {'color': 'rgb(249,123,114)'},
              'name': 'Declines',
              'type': 'bar',
              'x': array([datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0),
                          datetime.datetime(2022, 1, 6, 0, 0), ...,
                          datetime.datetime(2023, 6, 28, 0, 0),
                          datetime.datetime(2023, 6, 29, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object),
              'xaxis': 'x2',
              'y': array([10., 16., 44., ..., 32., 16.,  1.]),
              'yaxis': 'y2'},
             {'line': {'color': '#496CFD', 'width': 2},
              'mode': 'lines',
              'name': 'McClellan Oscillator',
              'type': 'scatter',
              'x': array([datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0),
                          datetime.datetime(2022, 1, 6, 0, 0), ...,
                          datetime.datetime(2023, 6, 28, 0, 0),
                          datetime.datetime(2023, 6, 29, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object),
              'xaxis': 'x3',
              'y': array([ 5.2 ,  4.35,  1.95, ..., -2.95, -2.4 ,  0.15]),
              'yaxis': 'y3'}],
    'layout': {'barmode': 'relative',
               'height': 900,
               'plot_bgcolor': '#F7F9F9',
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': '#566573', 'family': 'Arial', 'size': 16},
                         'text': 'SET50 Index Futures and Market Breadth Indicators'},
               'width': 1200,
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'matches': 'x3',
                         'range': [2023-05-26 22:15:04

###### **SET50 Index Futures and Moving Average**

In [ ]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET50_FUTURES_BREADTH['Date'].iloc[0], end=SET50_FUTURES_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET50_FUTURES_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET50_FUTURES_BREADTH['Date'],
                            open=SET50_FUTURES_BREADTH['Open'],
                            high=SET50_FUTURES_BREADTH['High'],
                            low=SET50_FUTURES_BREADTH['Low'],
                            close=SET50_FUTURES_BREADTH['Close'],
                            name='SET50 Index Futures'))

# Plot Moving Averages 20, 60, 200 days on the second row
fig1.add_trace(go.Scatter(name='20d MA',
                          x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['MA20'],
                          mode='lines', line=dict(color='#80ED7C', width=1.5)),
               row=2, col=1)
fig1.add_trace(go.Scatter(name='60d MA',
                          x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['MA60'],
                          mode='lines', line=dict(color='#51D64D', width=2.0)),
               row=2, col=1)
fig1.add_trace(go.Scatter(name='200d MA',
                          x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['MA200'],
                          mode='lines', line=dict(color='#20B71B', width=2.5)),
               row=2, col=1)

# Plot Ratio Advances / (Advances + Declines) on the third row
fig1.add_trace(go.Scatter(name='Ratio Adv-to-Adv&Dec',
                          x=SET50_FUTURES_BREADTH['Date'],
                          y=getEMA(100*(SET50_FUTURES_BREADTH['Advance']/(SET50_FUTURES_BREADTH['Advance']+SET50_FUTURES_BREADTH['Declines'])), 10),
                          mode='lines', line=dict(color='#496CFD', width=2)),
               row=3, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET50 Index Futures",
                  range=[800, 1000], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="Moving Averages", 
                  range=[0, 50], tickmode='array', tickvals=(0, 50, 25),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="Ratio Advance-to-Declines", 
                  range=[25, 75], tickmode='array', tickvals=(25, 75, 50),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=10, row=3, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET50 Index Futures and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])

###### **SET50 Index Futures, New High and New Low**

In [ ]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET50_FUTURES_BREADTH['Date'].iloc[0], end=SET50_FUTURES_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET50_FUTURES_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET50_FUTURES_BREADTH['Date'],
                            open=SET50_FUTURES_BREADTH['Open'],
                            high=SET50_FUTURES_BREADTH['High'],
                            low=SET50_FUTURES_BREADTH['Low'],
                            close=SET50_FUTURES_BREADTH['Close'],
                            name='SET50 Index Futures'))

# Plot New High and New Low 20 days on the second row
fig1.add_trace(go.Bar(name="New High 20d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['NH20'], 
                      marker_color='#80ED7C'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="New Low 20d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=(-1)*SET50_FUTURES_BREADTH['NL20'], 
                      marker_color='#EB7D7D'),
              row=2, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 20d',
                          x=SET50_FUTURES_BREADTH['Date'], y=getEMA((SET50_FUTURES_BREADTH['NH20'] - SET50_FUTURES_BREADTH['NL20']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=2, col=1)

# Plot New High and New Low 60 days on the third row
fig1.add_trace(go.Bar(name="New High 60d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['NH60'], 
                      marker_color='#51D64D'),
              row=3, col=1)
fig1.add_trace(go.Bar(name="New Low 60d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=(-1)*SET50_FUTURES_BREADTH['NL60'], 
                      marker_color='#E34545'),
              row=3, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 60d',
                          x=SET50_FUTURES_BREADTH['Date'], y=getEMA((SET50_FUTURES_BREADTH['NH60'] - SET50_FUTURES_BREADTH['NL60']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=3, col=1)


# Plot New High and New Low 250 days on the third row
fig1.add_trace(go.Bar(name="New High 250d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=SET50_FUTURES_BREADTH['NH250'], 
                      marker_color='#20B71B'),
              row=4, col=1)
fig1.add_trace(go.Bar(name="New Low 250d", 
                      x=SET50_FUTURES_BREADTH['Date'], y=(-1)*SET50_FUTURES_BREADTH['NL250'], 
                      marker_color='#E10404'),
              row=4, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 250d',
                          x=SET50_FUTURES_BREADTH['Date'], y=getEMA((SET50_FUTURES_BREADTH['NH250'] - SET50_FUTURES_BREADTH['NL250']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=4, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET50 Index Futures",
                  range=[800, 1000], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="NH NL 20 days", 
                  range=[-25, 25], tickmode='array', tickvals=(-25, 25, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="NH NL 60 days", 
                  range=[-25, 25], tickmode='array', tickvals=(-25, 25, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=3, col=1)

fig1.update_yaxes(title_text="NH NL 250 days", 
                  range=[-25, 25], tickmode='array', tickvals=(-25, 25, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=4, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET50 Index Futures and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])

##### **SET Visualization**

###### **SET Index, Advances-Declines, McClellan Oscillator**

In [ ]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET_INDEX_BREADTH['Date'].iloc[0], end=SET_INDEX_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET_INDEX_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET_INDEX_BREADTH['Date'],
                            open=SET_INDEX_BREADTH['Open'],
                            high=SET_INDEX_BREADTH['High'],
                            low=SET_INDEX_BREADTH['Low'],
                            close=SET_INDEX_BREADTH['Close'],
                            name='SET Index'))

# Calculate the total number of stocks
noStock = SET_Index.Advance[0] + SET_Index.Unchanged[0] + SET_Index.Declines[0]

# Plot Advances, Unchanged, and Declines on the second row
fig1.add_trace(go.Bar(name="Advances", 
                      x=SET_INDEX_BREADTH['Date'], y=100 * SET_INDEX_BREADTH['Advance'] / noStock, 
                      marker_color='rgb(128,186,90)'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="Unchanged", 
                      x=SET_INDEX_BREADTH['Date'], y=100 * SET_INDEX_BREADTH['Unchanged'] / noStock, 
                      marker_color='rgb(204,204,204)'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="Declines", 
                      x=SET_INDEX_BREADTH['Date'], y=100 * SET_INDEX_BREADTH['Declines'] / noStock, 
                      marker_color='rgb(249,123,114)'),
              row=2, col=1)
fig1.update_layout(barmode='relative')

# Plot McClellan Oscillator on the third row
fig1.add_trace(go.Scatter(name='McClellan Oscillator',
                          x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['McClellanOscillator'],
                          mode='lines', line=dict(color='#496CFD', width=2)),
               row=3, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET Index",
                  range=[1350, 1600], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="Advances/Unchanged/Declines", 
                  range=[0, 100], tickmode='array', tickvals=(0, 100, 50),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="McClellan Oscillator", 
                  range=[-100, 100], tickmode='array', tickvals=(-100, 100, 0.0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=10, row=3, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET Index and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])

###### **SET Index and Moving Average**

In [ ]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET_INDEX_BREADTH['Date'].iloc[0], end=SET_INDEX_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET_INDEX_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET_INDEX_BREADTH['Date'],
                            open=SET_INDEX_BREADTH['Open'],
                            high=SET_INDEX_BREADTH['High'],
                            low=SET_INDEX_BREADTH['Low'],
                            close=SET_INDEX_BREADTH['Close'],
                            name='SET Index'))

# Calculate the total number of stocks
noStock = SET_INDEX_BREADTH.Advance[0] + SET_INDEX_BREADTH.Unchanged[0] + SET_INDEX_BREADTH.Declines[0]

# Plot Moving Averages 20, 60, 200 days on the second row
fig1.add_trace(go.Scatter(name='20d MA',
                          x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['MA20'],
                          mode='lines', line=dict(color='#80ED7C', width=1.5)),
               row=2, col=1)
fig1.add_trace(go.Scatter(name='60d MA',
                          x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['MA60'],
                          mode='lines', line=dict(color='#51D64D', width=2.0)),
               row=2, col=1)
fig1.add_trace(go.Scatter(name='200d MA',
                          x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['MA200'],
                          mode='lines', line=dict(color='#20B71B', width=2.5)),
               row=2, col=1)

# Plot Ratio Advances / (Advances + Declines) on the third row
fig1.add_trace(go.Scatter(name='Ratio Adv-to-Adv&Dec',
                          x=SET_INDEX_BREADTH['Date'],
                          y=getEMA(100*(SET_INDEX_BREADTH['Advance']/(SET_INDEX_BREADTH['Advance']+SET_INDEX_BREADTH['Declines'])), 10),
                          mode='lines', line=dict(color='#496CFD', width=2)),
               row=3, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET Index",
                  range=[1350, 1600], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="Moving Averages", 
                  range=[0, 100], tickmode='array', tickvals=(0, 100, 50),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="Ratio Advance-to-Declines", 
                  range=[25, 75], tickmode='array', tickvals=(25, 75, 50),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=10, row=3, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET Index and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])

###### **SET Index, New High and New Low**

In [ ]:
# Create a date range from the start date to the end date
dt_all = pd.date_range(start=SET_INDEX_BREADTH['Date'].iloc[0], end=SET_INDEX_BREADTH['Date'].iloc[-1])

# Retrieve the dates that are in the original dataset
dt_obs = [d for d in pd.to_datetime(SET_INDEX_BREADTH['Date'])]

# Define dates with missing values
dt_breaks = [d.strftime("%Y-%m-%d") for d in dt_all if d not in dt_obs]

# Create a subplot figure with three rows and one column
fig1 = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.4, 0.2, 0.2, 0.2])

# Plot SET Index on the first row
fig1.add_trace(go.Candlestick(x=SET_INDEX_BREADTH['Date'],
                            open=SET_INDEX_BREADTH['Open'],
                            high=SET_INDEX_BREADTH['High'],
                            low=SET_INDEX_BREADTH['Low'],
                            close=SET_INDEX_BREADTH['Close'],
                            name='SET Index'))

# Plot New High and New Low 20 days on the second row
fig1.add_trace(go.Bar(name="New High 20d", 
                      x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['NH20'], 
                      marker_color='#80ED7C'),
              row=2, col=1)
fig1.add_trace(go.Bar(name="New Low 20d", 
                      x=SET_INDEX_BREADTH['Date'], y=(-1)*SET_INDEX_BREADTH['NL20'], 
                      marker_color='#EB7D7D'),
              row=2, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 20d',
                          x=SET_INDEX_BREADTH['Date'], y=getEMA((SET_INDEX_BREADTH['NH20'] - SET_INDEX_BREADTH['NL20']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=2, col=1)

# Plot New High and New Low 60 days on the third row
fig1.add_trace(go.Bar(name="New High 60d", 
                      x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['NH60'], 
                      marker_color='#51D64D'),
              row=3, col=1)
fig1.add_trace(go.Bar(name="New Low 60d", 
                      x=SET_INDEX_BREADTH['Date'], y=(-1)*SET_INDEX_BREADTH['NL60'], 
                      marker_color='#E34545'),
              row=3, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 60d',
                          x=SET_INDEX_BREADTH['Date'], y=getEMA((SET_INDEX_BREADTH['NH60'] - SET_INDEX_BREADTH['NL60']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=3, col=1)


# Plot New High and New Low 250 days on the third row
fig1.add_trace(go.Bar(name="New High 250d", 
                      x=SET_INDEX_BREADTH['Date'], y=SET_INDEX_BREADTH['NH250'], 
                      marker_color='#20B71B'),
              row=4, col=1)
fig1.add_trace(go.Bar(name="New Low 250d", 
                      x=SET_INDEX_BREADTH['Date'], y=(-1)*SET_INDEX_BREADTH['NL250'], 
                      marker_color='#E10404'),
              row=4, col=1)
fig1.update_layout(barmode='relative')
fig1.add_trace(go.Scatter(name='Diff. NH-NL 250d',
                          x=SET_INDEX_BREADTH['Date'], y=getEMA((SET_INDEX_BREADTH['NH250'] - SET_INDEX_BREADTH['NL250']), 2),
                          mode='lines', line=dict(color='black', width=1.2, dash='dot')),
               row=4, col=1)

# Update layout
fig1.update_layout(height=900, width=1200, 
                   showlegend=False, xaxis_rangeslider_visible=False, 
                   plot_bgcolor="#F7F9F9")

# Hide dates with no values
fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update y-axis titles and fonts
fig1.update_yaxes(title_text="SET Index",
                  range=[1350, 1600], tickmode='array',
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=6, row=1, col=1)

fig1.update_yaxes(title_text="NH NL 20 days", 
                  range=[-50, 50], tickmode='array', tickvals=(-50, 50, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=2, col=1)

fig1.update_yaxes(title_text="NH NL 60 days", 
                  range=[-50, 50], tickmode='array', tickvals=(-50, 50, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=3, col=1)

fig1.update_yaxes(title_text="NH NL 250 days", 
                  range=[-50, 50], tickmode='array', tickvals=(-50, 50, 0),
                  title_font=dict(size=12, family='Arial', color='#566573'), 
                  tickfont=dict(size=12, family='Arial', color='#566573'), 
                  title_standoff=15, row=4, col=1)

# Update x-axis label
fig1.update_xaxes(title_font=dict(size=16, family='Arial', color='#566573'), 
                  row=3, col=1)

# Update the overall layout title
fig1.update_layout(title='SET Index and Market Breadth Indicators', 
                   title_font=dict(size=16, family='Arial', color='#566573'), 
                   showlegend=True)

# Set the date range
start_date = dt.datetime.today() - dt.timedelta(int(365/2))
end_date = dt.datetime.today() + dt.timedelta(hours=7)
fig1.update_layout(xaxis_range=[start_date, end_date])